# ButtPlug Wars

## Setup

In [ ]:
import * as hardhat from 'hardhat'
import '@nomiclabs/hardhat-ethers'
import { ethers } from 'hardhat'
import { smock } from '@defi-wonderland/smock'

import * as bn from './utils/bn'
import * as evm from './utils/evm'
import * as wallet from './utils/wallet'
import * as contracts from './utils/contracts'

import {getMainnetSdk} from '@dethcrypto/eth-sdk-client';

In [ ]:
const WETH_9 = '0xB4FBF271143F4FBf7B91A5ded31805e42b2208d6';
const KEEP3R = '0x145d364e193204f8Ff0A87b718938406595678Dd';
const KP3R_LP = '0x78958e8e9C54d9aA56eDED102097E73ef9c26411';
const SUDOSWAP_FACTORY = '0xF0202E9267930aE942F0667dC6d805057328F6dC';
const SUDOSWAP_XYK_CURVE = '0x02363a2F1B2c2C5815cb6893Aa27861BE0c4F760';

In [ ]:
// FORK
let blockNumber = 8066864

await evm.reset({
    jsonRpcUrl: process.env['GOERLI_RPC'],
    blockNumber
})

const provider = ethers.provider

In [ ]:
// Players
const signer = await ethers.getSigner()
await wallet.fund(signer.address, bn.toUnit(10))

In [ ]:
// // Fork
// const gameAbi = await contracts.getContractAbi('./out/ButtPlugWars.sol/ButtPlugWars.json')
// const game = await ethers.getContractAt(gameAbi, '0x48c8c199ccdb7c1b7d3b41ee510ea2d4c65dca2c')

In [ ]:
// Deployment
const chess = await contracts.deploy(signer, './out/ChessForTest.sol/ChessForTest.json', [])

const args = [chess.address, WETH_9, KEEP3R, KP3R_LP, SUDOSWAP_FACTORY, SUDOSWAP_XYK_CURVE]
const game = await contracts.deploy(signer, './out/ButtPlugWars.sol/ButtPlugWars.json', args)

### Event start

In [ ]:
await evm.advanceTimeAndBlock(86400*10)
const tx = await game.startEvent();

### Player Badge minting

In [ ]:
await chess.totalSupply()

In [ ]:
// pre-genesis tokens
await chess.setApprovalForAll(game.address, true)

await game.mintPlayerBadge(0, 0, {value: bn.toUnit(1)})
const tx = await game.mintPlayerBadge(1, 1, {value: bn.toUnit(1)})

In [ ]:
const two = await game['getBadgeId(uint256)'](2)

### ButtPlug Badge minting

In [ ]:
const buttPlug = await contracts.deploy(signer, './out/ButtPlugForTest.sol/ButtPlugForTest.json', [])

const tx = await game.mintButtPlugBadge(buttPlug.address)

### Game start

In [ ]:
await evm.advanceTimeAndBlock(14 * 86400)
const tx = await game.pushLiquidity()

## Gameplay

In [ ]:
const { sudoPool, keep3r } = getMainnetSdk(signer)
const officialSudoPool = sudoPool.attach(await game.SUDOSWAP_POOL())
const testKeep3r = keep3r.attach(KEEP3R)

In [ ]:
const kp3rV1 = await testKeep3r.keep3rV1()
await testKeep3r.bond(kp3rV1,0)
await testKeep3r.activate(kp3rV1)
'activate keeper'

### E2E environment

In [ ]:
await game['voteButtPlug(address,uint256)'](buttPlug.address, 1);
await game['voteButtPlug(address,uint256[])'](buttPlug.address, [two]);

const board = await chess.board()
await buttPlug.setMove(board,1)
await buttPlug.setDepth(3)
await evm.advanceTimeAndBlock(5*86400)
const tx = await game.executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

In [ ]:
const tx = await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

### Post Genesis badges

In [ ]:
const receipt = await tx.wait()
const postGenesisToken = receipt.events[2].topics[3]

In [ ]:
await game.mintPlayerBadge(postGenesisToken, 1, {value: bn.toUnit(0.1)})
const postGenBadge = await game['getBadgeId(uint256)'](3)

## Token URIs

In [ ]:
const scoreboard = await game.tokenURI(0)

In [ ]:
JSON.parse(atob(scoreboard.substring(29,1e6)))

In [ ]:
JSON.parse(JSON.stringify(await game.tokenURI(1)))

In [ ]:
const buttPlugBadge = await game['getBadgeId(address)'](buttPlug.address)
JSON.parse(await game.tokenURI(buttPlugBadge))

## Prize claiming

In [ ]:
const tx = await game.unbondLiquidity()

await contracts.logTx(tx)

In [ ]:
const tx = await game['claimPrize(uint256)'](5)
// const tx = await game['claimPrize(uint256[])']([0,1,2,3,4])

await contracts.logTx(tx)

### Prize ceremony

In [ ]:
await evm.advanceTimeAndBlock(14*86400)
const tx = await game.withdrawLiquidity()

await contracts.logTx(tx)

In [ ]:
const tx = await game.withdrawPrize()

await contracts.logTx(tx)